<a href="https://colab.research.google.com/github/chueneelvin/Databricks/blob/main/RAG_with_Lanchain_and_Neo4j_(knowledge_graph_db).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required libraries

In [2]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 31.6 MB/s eta 0:00:00
   ━━

# Import the required libraries

In [15]:
from langchain_core.runnables import RunnableBranch,RunnableLambda,RunnableParallel,RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableField
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
import os
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

# Configure secrets and API keys

In [4]:
# Get the API key from user data
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["NEO4J_URI"] = userdata.get('NEO4J_URI')
os.environ["NEO4J_USERNAME"] = userdata.get('NEO4J_USERNAME')
os.environ["NEO4J_PASSWORD"] = userdata.get('NEO4J_PASSWORD')

#Load the data

In [5]:
raw_documents = WikipediaLoader(query="Nelson Mandela").load()
raw_documents[22]

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Document(metadata={'title': 'John Carlin (journalist)', 'summary': 'John Carlin (born 12 May 1956) is a British journalist and author, who deals with both sports and politics. His book Playing the Enemy: Nelson Mandela and the Game that Made a Nation, about former South African president Nelson Mandela, is the basis for the 2009 film Invictus.', 'source': 'https://en.wikipedia.org/wiki/John_Carlin_(journalist)'}, page_content='John Carlin (born 12 May 1956) is a British journalist and author, who deals with both sports and politics. His book Playing the Enemy: Nelson Mandela and the Game that Made a Nation, about former South African president Nelson Mandela, is the basis for the 2009 film Invictus.\n\n\n== Personal life ==\nCarlin was born to a Scottish father and Spanish mother. He spent the first three years of his life in North London, before moving to Buenos Aires, Argentina, due to his father\'s posting to the British Embassy.\nAfter returning to England, he was educated at St. G

In [6]:
len(raw_documents)

24

# Split the documents into chunks

In [7]:
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents)
documents[:3]

[Document(metadata={'title': 'Nelson Mandela', 'summary': 'Nelson Rolihlahla Mandela ( man-DEH-lə; Xhosa: [xolíɬaɬa mandɛ̂ːla]; born Rolihlahla Mandela; 18 July 1918 – 5 December 2013) was a South African anti-apartheid activist, politician, and statesman who served as the first president of South Africa from 1994 to 1999. He was the country\'s first black head of state and the first elected in a fully representative democratic election. His government focused on dismantling the legacy of apartheid by fostering racial reconciliation. Ideologically an African nationalist and socialist, he served as the president of the African National Congress (ANC) party from 1991 to 1997.\nA Xhosa, Mandela was born into the Thembu royal family in Mvezo, South Africa. He studied law at the University of Fort Hare and the University of Witwatersrand before working as a lawyer in Johannesburg. There he became involved in anti-colonial and African nationalist politics, joining the ANC in 1943 and co-foun

# Instantiate an LLM Model

In [8]:
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

# Instantiate LLM Graph transformer

In [9]:
llm_transformer = LLMGraphTransformer(llm=llm)

# Converts documents into graph documents

In [11]:
graph_documents = llm_transformer.convert_to_graph_documents(documents[:4])

In [12]:
graph_documents[0]

GraphDocument(nodes=[Node(id='Nelson Rolihlahla Mandela', type='Person'), Node(id='South Africa', type='Country'), Node(id='African National Congress', type='Political party'), Node(id='University Of Fort Hare', type='University'), Node(id='University Of Witwatersrand', type='University'), Node(id='Johannesburg', type='City'), Node(id='Thembu Royal Family', type='Family'), Node(id='Mvezo', type='Place'), Node(id='Transvaal Branch', type='Organization'), Node(id='Umkhonto We Sizwe', type='Organization'), Node(id='South African Communist Party', type='Political party'), Node(id='United States', type='Country'), Node(id='Robben Island', type='Prison'), Node(id='Pollsmoor Prison', type='Prison'), Node(id='Victor Verster Prison', type='Prison'), Node(id='F. W. De Klerk', type='Person')], relationships=[Relationship(source=Node(id='Nelson Rolihlahla Mandela', type='Person'), target=Node(id='South Africa', type='Country'), type='BIRTHPLACE'), Relationship(source=Node(id='Nelson Rolihlahla Man

# Load graph documents into graph db

In [13]:
graph = Neo4jGraph()
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)
graph

In [16]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [18]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [19]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [20]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))